### Ler repositório

In [1]:
!git clone https://github.com/unclecode/crawl4ai.git

Cloning into 'crawl4ai'...
remote: Enumerating objects: 10157, done.
remote: Counting objects: 100% (2492/2492), done.
remote: Compressing objects: 100% (529/529), done.
remote: Total 10157 (delta 2250), reused 1963 (delta 1963), pack-reused 7665 (from 1)
Receiving objects: 100% (10157/10157), 144.05 MiB | 23.05 MiB/s, done.
Resolving deltas: 100% (6529/6529), done.


### Importar token

#### Keggel

In [ ]:
# Importar token de acesso do huggingface
import os
from kaggle_secrets import UserSecretsClient
secret_label = "HF_TOKEN"
secret_value = UserSecretsClient().get_secret(secret_label)


os.environ['HF_TOKEN'] = secret_value

#### Colab

In [2]:
from google.colab import userdata
import os

hf_token = userdata.get('HF_TOKEN')

os.environ['HF_TOKEN'] = hf_token

# Processar com o modelo 'Qwen/Qwen2.5-Coder-7B-Instruct'


## Definir prompt de Sistema

In [3]:
### Conteudo a ser incluido como prompt de sistema
system_content = f"""Você é um Engenheiro de Software de 160 de QI e vai analizar um repositorio e tentar identificar padrões arquiteturais de software nele.
"O que é um padrão arquitetural?

Descreve uma solução arquitetural para problemas
recorrentes em sistemas software que se
apresentam em contextos específicos.

A solução arquitetural do padrão define:

* as estruturas de software (componentes,
serviços, módulos)
* responsabilidades e relações entre essas
estruturas
* formas que as estruturas colaboram para
solucionar o problema"
Você vai receber os textos transcritos de arquivos de código do projeto e analizá-los junto com a árvore de diretórios e tentar encontrar estruturas recorrentes na organização do sistema que indiquem o uso de determinados padrões arquiteturais.
Você deve informar quais os principais padrões encontrados e dar uma justificativa para cada um.
Suas respostas devem ser em português.
Para se guiar procure principalmente por indícios das seguintes arquiteturas:
*Layers
*Pipe-Filter
*Cliente-Servidor
*Peer-to-Peer
*Service-Oriented Arquiteture
*Publish-Subscribe
*Data-Model
*Shared-Data
*Microservices
*Blackboard
"""

## 1. Um prompt para os arquivos mais importantes de uma só vez

### Ler conteúdo dos arquivos para inserir no prompt de usuario

In [4]:
import os

IMPORTANT_FILES_ROOT = [
    "Dockerfile",
    "docker-compose.yml",
    "pyproject.toml"
]
IMPORTANT_FILES_CRAWL = [
    "async_webcrawler.py",
    "cli.py",
    "config.py",
    "extraction_strategy.py",
    "browser_manager.py",
    "models.py",
    "model_loader.py",
    "async_dispatcher.py"
]
IMPORTANT_FILES_DEPLOY_DOCKER = [
    "schemas.py",
    "server.py",
    "api.py"
]

# string para guardar conteudo dos arquivos
total_content = "Analise os padrões arquiteturais dos seguintes arquivos: "

print("Lendo conteudo dos arquivos:")

root_dir = "crawl4ai"
for file_name in IMPORTANT_FILES_ROOT:
  file_path = os.path.join(root_dir, file_name)
  try:
    with open(file_path, 'r') as f:
      content = f.read()
      total_content = f"\n{file_path}\n\nConteúdo:\n{content}"
  except Exception as e:
            print(f"Could not read {file_path}: {e}")
            pass

for file_name in IMPORTANT_FILES_CRAWL:
  file_path = os.path.join(root_dir, "crawl4ai", file_name)
  try:
    with open(file_path, 'r') as f:
      content = f.read()
      total_content = f"\n{file_path}\n\nConteúdo:\n{content}"
  except Exception as e:
            print(f"Could not read {file_path}: {e}")
            pass

for file_name in IMPORTANT_FILES_DEPLOY_DOCKER:
  file_path = os.path.join(root_dir, "deploy", "docker", file_name)
  try:
    with open(file_path, 'r') as f:
      content = f.read()
      total_content = f"\n{file_path}\n\nConteúdo:\n{content}"
  except Exception as e:
            print(f"Could not read {file_path}: {e}")
            pass



Lendo conteudo dos arquivos:


### Rodar o modelo usando Inference Provider do Hugging Face

In [5]:
from openai import OpenAI

# Inferencia remota via provedores de inferencia
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)




# String para guardar respostas
generated_text = ""

messages_ = [
  {"role": "system", "content": system_content},
  {"role": "user", "content": total_content},
]

try:
    # Gerar resposta
    response = client.chat.completions.create(
        model="Qwen/Qwen2.5-Coder-7B-Instruct",
        messages=messages_
    )
    # Extrair texto gerado
    generated_text = response.choices[0].message.content

except Exception as e:
    generated_text = f"--- Erro ao processar ---\n\n {e}"


# Salvar respostas em um arquivo
with open('respostasP1.md', 'a') as f:
    file_anwser_content = f"### --- RESPOSTA PARA O PROMPT COM OS ARQUIVOS MAIS IMPORTANTES ---\n{generated_text}\n\n"
    f.write(file_anwser_content)


print("Respostas do modelo salvas em 'respostasP1.md'.")



Respostas do modelo salvas em 'respostasP1.md'.


## 2. Um prompt para cada arquivo por vez

### Listar os arquivos dentro do repositório 'crawl4ai'


In [6]:
import os

root_dir = 'crawl4ai'
file_paths = []

print(f"Listing files in '{root_dir}':")
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        full_path = os.path.join(dirpath, filename)
        file_paths.append(full_path)
        # print(full_path)

print(f"\nTotal files found: {len(file_paths)}")

Listing files in 'crawl4ai':

Total files found: 739


### Ler o conteúdo de cada arquivo identificado no repositório. Priorizaremos arquivos de código para a leitura do modelo.


In [7]:
import os

TEXT_CODE_EXTENSION = '.py'

# Dicionario vazio para guardar conteudo dos arquivos
file_contents = {}
read_files_count = 0

print("Lendo conteudo dos arquivos:")
for file_path in file_paths:
    # Verficar extensão do arquivo

    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension == TEXT_CODE_EXTENSION:
        try:
            with open(file_path, 'r') as f:
                content = f.read()
                file_contents[file_path] = content
                read_files_count += 1
        except Exception as e:
            print(f"Nao foi possivel ler {file_path}: {e}")
            pass

# Exibir o número total de arquivos dos quais o conteudo foi lido com sucesso
print(f"\nConteudo dos arquivos de texto/codigo: {read_files_count}.")
print(f"{len(file_contents)} armazenados no dicionario 'file_contents'.")

Lendo conteudo dos arquivos:

Conteudo dos arquivos de texto/codigo: 323.
323 armazenados no dicionario 'file_contents'.


### Rodar o modelo usando Inference Provider do Hugging Face

In [8]:
from openai import OpenAI

# Inferencia remota via provedores de inferencia
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)




# Lista para guardar respostas
all_responses = []

for nome_arq, conteudo_arq in file_contents.items():
    # Criando mensagem para o modelo
    prompt_content = f"Analise os padrões arquiteturais do seguinte arquivo: {nome_arq}\n\nConteúdo:\n{conteudo_arq[:2000]}"

    messages_ = [
      {"role": "system", "content": system_content},
      {"role": "user", "content": prompt_content},
    ]

    try:
        # Gerar resposta
        response = client.chat.completions.create(
            model="Qwen/Qwen2.5-Coder-7B-Instruct",
            messages=messages_
        )
        # Extrair texto gerado
        generated_text = response.choices[0].message.content
        all_responses.append(f"### --- RESPOSTA PARA {nome_arq} ---\n{generated_text}\n\n")

    except Exception as e:
        all_responses.append(f"--- Erro ao processar {nome_arq}: {e} ---\n\n")


# Salvar respostas em um arquivo
with open('respostasP2.md', 'a') as f:
    for res in all_responses:
        f.write(res)


print(f"Respostas do modelo salvas em 'respostasP2.md'. Arquivos processados: {len(all_responses)}")



Respostas do modelo salvas em 'respostasP2.md'. Arquivos processados: 323
